In [134]:
# Import packages
import os
import warnings
import cartopy.crs as ccrs
from cartopy import feature as cf
import matplotlib.pyplot as plt
import numpy.ma as ma
import xarray as xr
import rioxarray as rxr
import glob as glob
import pandas as pd
import numpy as np
warnings.simplefilter('ignore')

In [73]:
year=[str(i).zfill(2) for i in range(1,21)]
day=[str(i).zfill(3) for i in range(1,367)]
print("../../scratch/nishant/MODIS_data/MOD11C1.A20"+year[0]+day[0]+"*.hdf")

../../scratch/nishant/MODIS_data/MOD11C1.A2001001*.hdf


In [135]:
years = [str(i) for i in range(2001,2021)]
years_int = [int(i) for i in range(2001,2021)]
# year=[str(i).zfill(2) for i in range(1,21)]
day=[str(i).zfill(3) for i in range(1,367)]
start_date = '2001-01-01'
end_date = '2020-12-31'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
timeseries=pd.DataFrame(date_range)
cnt=timeseries.count()
q=0

compiled=[None]*20
for y in range(0,20):
    
    # To get year (integer input) from the user
    # year = int(input("Enter a year: "))
    
    # divided by 100 means century year (ending with 00)
    # century year divided by 400 is leap year
    if (years_int[y] % 400 == 0) and (years_int[y] % 100 == 0):
        LST=[None]*366
        print(years[y])
        # print('This is a leap year')
        for d in range(0,366):
            try:
                file=glob.glob("../../scratch/nishant/MODIS_data/MOD11C1.A"+years[y]+day[d]+"*.hdf")
                LS = rxr.open_rasterio(file[0], masked=True)
                LS = LS["LST_Day_CMG"].squeeze().sel(x=slice('72.75','73.00'), y=slice('19.35','18.85'))*0.02
                LS = LS.rename({'x':'lon'}).rename({'y':'lat'})
                LS = LS.reset_coords('band', drop=True).reset_coords('spatial_ref', drop=True)
                LST[d] = LS.assign_coords({'time':date_range[q]}).expand_dims('time').to_dataset()
                q=q+1
            except:
                LST[d]=xr.full_like(LST[0],np.nan)
                var=LST[d].drop_dims('time').assign_coords({'time':date_range[q]}).expand_dims('time')
                LST[d]['time']=var['time']
                q=q+1
                continue
        compiled[y]=LST
                
    # not divided by 100 means not a century year
    # year divided by 4 is a leap year
    elif (years_int[y] % 4 ==0) and (years_int[y] % 100 != 0):
        LST=[None]*366
        print(years[y])
        # print('This is a leap year')
        for d in range(0,366):
            try:
                file=glob.glob("../../scratch/nishant/MODIS_data/MOD11C1.A"+years[y]+day[d]+"*.hdf")
                LS = rxr.open_rasterio(file[0], masked=True)
                LS = LS["LST_Day_CMG"].squeeze().sel(x=slice('72.75','73.00'), y=slice('19.35','18.85'))*0.02
                LS = LS.rename({'x':'lon'}).rename({'y':'lat'})
                LS = LS.reset_coords('band', drop=True).reset_coords('spatial_ref', drop=True)
                LST[d] = LS.assign_coords({'time':date_range[q]}).expand_dims('time').to_dataset()
                q=q+1
            except:
                LST[d]=xr.full_like(LST[0],np.nan)
                var=LST[d].drop_dims('time').assign_coords({'time':date_range[q]}).expand_dims('time')
                LST[d]['time']=var['time']
                q=q+1
                continue
        compiled[y]=LST
    
    # if not divided by both 400 (century year) and 4 (not century year)
    # year is not leap year
    else:
        LST=[None]*365
        print(years[y])
        for d in range(0,365):
            try:
                file=glob.glob("../../scratch/nishant/MODIS_data/MOD11C1.A"+years[y]+day[d]+"*.hdf")
                LS = rxr.open_rasterio(file[0], masked=True)
                LS = LS["LST_Day_CMG"].squeeze().sel(x=slice('72.75','73.00'), y=slice('19.35','18.85'))*0.02
                LS = LS.rename({'x':'lon'}).rename({'y':'lat'})
                LS = LS.reset_coords('band', drop=True).reset_coords('spatial_ref', drop=True)
                LST[d] = LS.assign_coords({'time':date_range[q]}).expand_dims('time').to_dataset()
                q=q+1
            except:
                LST[d]=xr.full_like(LST[0],np.nan)
                var=LST[d].drop_dims('time').assign_coords({'time':date_range[q]}).expand_dims('time')
                LST[d]['time']=var['time']
                q=q+1
                continue
        compiled[y]=LST

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [136]:
year_concat=[None]*20
for i in range(0,20):
    year_concat[i] = xr.concat(compiled[i], dim='time')

In [137]:
final_datas=xr.concat(year_concat,dim='time')

In [138]:
final_datas.to_netcdf('../../scratch/nishant/MODIS_data/All_Mumbai_MODIS_LST_Day_CMG.nc', mode='w')

In [130]:
new_LST=xr.open_dataset('../../scratch/nishant/MODIS_data/Mumbai_MODIS_LST_Day_CMG.nc')['LST_Day_CMG']

In [129]:
newnew_LST.groupby('time.month').max()

DataArrayGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [131]:
years = [str(i) for i in range(2001,2021)]
years_int = [int(i) for i in range(2001,2021)]
# year=[str(i).zfill(2) for i in range(1,21)]
day=[str(i).zfill(3) for i in range(1,367)]
start_date = '2001-01-01'
end_date = '2001-12-31'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
final_dat = [None]*20
LST=[None]*365
q=0
print(years[0])
for d in range(0,365):
    try:
        file=glob.glob("../../scratch/nishant/MODIS_data/MOD11C1.A"+years[0]+day[d]+"*.hdf")
        LS = rxr.open_rasterio(file[0], masked=True)
        LS = LS["LST_Day_CMG"].squeeze().sel(x=slice('72.75','72.95'), y=slice('19.2','18.9'))*0.02
        LS = LS.rename({'x':'lon'}).rename({'y':'lat'})
        LS = LS.reset_coords('band', drop=True).reset_coords('spatial_ref', drop=True)
        LST[d] = LS.assign_coords({'time':date_range[q]}).expand_dims('time')
        q=q+1
    except:
        continue
        q=q+1
final_dat[0]=LST

2001


In [136]:
final_dat[0][364]

<xarray.DataArray 'LST_Day_CMG' (time: 1, lat: 6, lon: 4)>
array([[[303.  , 303.44, 301.82, 303.88],
        [302.52, 304.08, 303.7 , 303.1 ],
        [302.1 , 305.54, 305.1 , 301.62],
        [303.16, 304.2 , 304.44, 302.02],
        [304.02, 304.02, 304.02,    nan],
        [   nan,    nan, 299.84, 302.68]]])
Coordinates:
  * lat      (lat) float64 19.17 19.12 19.08 19.02 18.97 18.92
  * lon      (lon) float64 72.78 72.83 72.88 72.93
  * time     (time) datetime64[ns] 2001-12-14

In [177]:
# Python program to check if year is a leap year or not
year = 2004

# To get year (integer input) from the user
# year = int(input("Enter a year: "))

# divided by 100 means century year (ending with 00)
# century year divided by 400 is leap year
if (year % 400 == 0) and (year % 100 == 0):
    print("{0} is a leap year".format(year))

# not divided by 100 means not a century year
# year divided by 4 is a leap year
elif (year % 4 ==0) and (year % 100 != 0):
    print("{0} is a leap year".format(year))

# if not divided by both 400 (century year) and 4 (not century year)
# year is not leap year
else:
    print("{0} is not a leap year".format(year))

2004 is a leap year


In [118]:
# years = [str(i) for i in range(2001,2021)]
# day=[str(i).zfill(3) for i in range(1,367)]
start_date = '2001-01-01'
end_date = '2001-01-31'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
day=[str(i).zfill(3) for i in range(1,31)]
fd=[None]*20
LS=[None]*31
q=0
for i in range(0,31):
    try:
        files=glob.glob("../../scratch/nishant/MODIS_data/MOD11C1.A2001"+day[i]+"*.hdf")
        L = rxr.open_rasterio(files[0], masked=True)
        L = L["LST_Day_CMG"].squeeze().sel(x=slice('72.75','72.95'), y=slice('19.2','18.9'))*0.02
        L = L.rename({'x':'lon'}).rename({'y':'lat'})
        L = L.reset_coords('band', drop=True).reset_coords('spatial_ref', drop=True)
        LS[i] = L.assign_coords({'time':date_range[q]}).expand_dims('time').to_dataset()
        q=q+1
    except:
        LS[i]=xr.full_like(LS[0],np.nan)
        var=LS[i].drop_dims('time').assign_coords({'time':date_range[q]}).expand_dims('time')
        LS[i]['time']=var['time']
        q=q+1
        continue
fd[0]=LS

In [119]:
a=xr.concat(fd[0], dim='time')
a.isel(time=1)

<xarray.Dataset>
Dimensions:      (lat: 6, lon: 4)
Coordinates:
  * lat          (lat) float64 19.17 19.12 19.08 19.02 18.97 18.92
  * lon          (lon) float64 72.78 72.83 72.88 72.93
    time         datetime64[ns] 2001-01-02
Data variables:
    LST_Day_CMG  (lat, lon) float64 nan nan nan nan nan ... nan nan nan nan nan

In [75]:
day=[str(i).zfill(3) for i in range(1,367)]
files=[None]*365
for i in range(0,365):
    files[i]=glob.glob("../../scratch/nishant/MODIS_data/MOD11C1.A2001"+day[i]+"*.hdf")
files

[['../../scratch/nishant/MODIS_data/MOD11C1.A2001001.061.2020081201310.hdf'],
 ['../../scratch/nishant/MODIS_data/MOD11C1.A2001002.061.2020081210234.hdf'],
 ['../../scratch/nishant/MODIS_data/MOD11C1.A2001003.061.2020081235725.hdf'],
 ['../../scratch/nishant/MODIS_data/MOD11C1.A2001004.061.2020082001418.hdf'],
 ['../../scratch/nishant/MODIS_data/MOD11C1.A2001005.061.2020082030329.hdf'],
 ['../../scratch/nishant/MODIS_data/MOD11C1.A2001006.061.2020082031541.hdf'],
 ['../../scratch/nishant/MODIS_data/MOD11C1.A2001007.061.2020082070207.hdf'],
 ['../../scratch/nishant/MODIS_data/MOD11C1.A2001008.061.2020082070134.hdf'],
 ['../../scratch/nishant/MODIS_data/MOD11C1.A2001009.061.2020082154800.hdf'],
 ['../../scratch/nishant/MODIS_data/MOD11C1.A2001010.061.2020082173121.hdf'],
 ['../../scratch/nishant/MODIS_data/MOD11C1.A2001011.061.2020082182425.hdf'],
 ['../../scratch/nishant/MODIS_data/MOD11C1.A2001012.061.2020082201101.hdf'],
 ['../../scratch/nishant/MODIS_data/MOD11C1.A2001013.061.2020082

In [16]:
len(date_range)

31

In [18]:
day[30]

'365'